## 1. SGNS

### How to build the ingroup/outgroup dimensions?
Build one vector for each dimension based on a set of words *L* from the replacement experiments coded as "ingroup" or "outgroup".

* Top word (|*L*| = 1), given some measure, e.g. frequency or "keyness" (see below)
* Top *k* words (|*L*| = *k*), given some measure, e.g. frequency or "keyness"
* Manual selection of *L*; can be more or less motivated by some measure, e.g. frequency or "keyness"

Given *L* the association with a dog whistle expression (DWE) over time can be measured by:

* The similarity of the vector of the DWE at *t* and the mean of vectors of words in *L* (at *t*)
* The mean of similarity of the vector of the DWE (at *t*) and the vector of each word in *L* (given this latter approach, it is possible to use a *T*-test to compare if there is a difference in in- vs out-group association; the methodological implications of this shold be though through, though).


### Morphology
The replacement tests are done with specific forms, e.g. plural of *globalist*: "globalister". Hence, the responses (replacements) will typically align with the target form: "elitist*er*". Also *förortsgäng* is a collective noun with its replacement typically being plural ("kriminella ungdom*ar*"). These considerations poses a question about how (if at all) to expand *L* by including morphologically related forms when building the in/outgroup dimensions. 

### Should multiple words be combined (e.g., *skicka tillbaka*)? If so, how? 
(https://www.baeldung.com/cs/sentence-vectors-word2vec)

For the SGNS models, it is easier if we ignore phrases and exclude "stopwords". But otherwise, the following methods can be considered to combine vectors:

* addition
* averaging
* weigthed averaging

### How to avoid overlap of dimensions? Keyness?
For example, for *förortsgäng*, *kriminell* is a frequent term in replacements of both the ingroup and the outgroup. To handle this, some kind of keyword methodology can be considered (cf. corpus linguistics and meausures of "keyness", that considers the probability of term *t* in an ingoup vs. en outgroup "corpus").

## 2. AVG BERT
### Scope
Complete or partial? I.e., all `meaning = 1` or just a selection? (a question similar to determine *L* above)

### Out of vocabulary
Will out of vocabulary be a problem for a non-domain adapted BERT? Will the tokenization of BERT "take care" of this? I.e., given the short "sentences" in replacements, will BERT be able to differentiate in/outgroup dimensions? This is the question of Hertzberg (et al.). And the answer was "yes" (but note fine-tuned vs. not fine-tuned). 

## 3. CLT BERT
The same questions of scope and OOV as for AVG BERT applies for CLT BERT. Additional options for how to relate in/outgroup dimensions to clusters include:
* Use the centroids of clusters
* Use jaccard similarity with top terms of clusters, given some weigthed frequency distribution; conider, for example, class-based TF-IDF. 

What about *hjälpa på plats*?

In [1]:
import os
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from pathlib import Path
from collections import Counter
from util import load_metric, read_util

from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import ttest_ind

## Data

In [2]:
with open(Path("../data/utils/stopwords-sv.txt")) as f:
    stopwords = [sw.replace("\n", "") for sw in f.readlines()]

In [3]:
path_to_data = Path("/home/max/Documents/research/replacement_data/panel_wide_onlyreplace.csv")
replacement_data = pd.read_csv(path_to_data, sep="\t")

In [4]:
#replacement_data

In [5]:
replacement_data = replacement_data.drop(0)

In [6]:
#replacement_data

In [7]:
#df= df.applymap(lambda s:s.lower() if type(s) == str else s)
replacement_data = replacement_data.applymap(lambda s: s.lower() if type(s) == str else s)

In [8]:
replacement_data

,forortsgang_text_w1,forortsgang_w1_C,forortsgang_text_w2,forortsgang_w2_C,aterinvandring_text_w1,aterinvandring_w1_C,aterinvandring_text_w2,aterinvandring_w2_C,berikar_text_w1,berikar_w1_C,...,globalister_text_w2,globalister_w2_C,ordningreda_text_w1,ordningreda_w1_C,ordningreda_text_w2,ordningreda_w2_C,sjalvstandig_text_w1,sjalvstandig_w1_C,sjalvstandig_text_w2,sjalvstandig_w2_C
1,NaN,NaN,unga från ytterområden,2.0,NaN,NaN,repatriering,2.0,NaN,NaN,...,världselit,1.0,NaN,NaN,-99,NaN,NaN,NaN,egen policy utomlands.,2.0
2,NaN,NaN,gäng,2.0,NaN,NaN,hjälp att flytta tillbaka till hemlandet,2.0,NaN,NaN,...,-99,NaN,NaN,NaN,en tydlig riktning i flyktingpolitiken,2.0,NaN,NaN,egen utrikespolitik,2.0
3,"småpojkar,",2.0,svartskallar,1.0,"repatriering, tvångsförflyttning",1,repatriering,2.0,"tillför värde, förbättrar",2.0,...,landsförrädare,2.0,bryta mot asylrätten,1.0,minska flyktingströmmen,1.0,"neutral, ej avhängigt andras agendor,",2.0,neutral,2.0
4,NaN,NaN,gängkriminiella sammanslutningar,2.0,NaN,NaN,att få invandrare att återvända till sina heml...,2.0,NaN,NaN,...,företrädare med ett internationellt perspektiv,2.0,NaN,NaN,struktur och tydlighet i migrationspolitiken,2.0,NaN,NaN,utrikespolitik fri från yttre påverkan,2.0
5,NaN,NaN,ungdomsgrupper från förorterna,2.0,NaN,NaN,att få personer att återvända till de länder f...,2.0,NaN,NaN,...,personer som inte tycker att det är viktigt me...,2.0,NaN,NaN,släppa in färre invandrare i sverige och att a...,1.0,NaN,NaN,utrikespolitik som inte ska påverkas av andra ...,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3238,NaN,NaN,kriminalitet,2.0,NaN,NaN,indraget uppehållstillstånd,1.0,NaN,NaN,...,-99,NaN,NaN,NaN,begränsad,1.0,NaN,NaN,-99,NaN
3239,NaN,NaN,gäng med ungdomar i orter kring de större städ...,2.0,NaN,NaN,att invandrarna flyttar tillbaka hem,2.0,NaN,NaN,...,ser hela världen hänga ihop,2.0,NaN,NaN,styra upp flyktingpolitiken,2.0,NaN,NaN,själv bestämma över utrikespolitiken,2.0
3240,NaN,NaN,ungdomar från socialt utsatta områden,2.0,NaN,NaN,att invandrare flyttar tillbaka till sina ursp...,2.0,NaN,NaN,...,-99,NaN,NaN,NaN,-99,NaN,NaN,NaN,egen utrikespolitik,2.0


In [9]:
set(w.split("_")[0] for w in replacement_data.columns)

{'aterinvandring',
 'berikar',
 'forortsgang',
 'globalister',
 'ordningreda',
 'sjalvstandig'}

## Functions

In [10]:
def inspect(
    df,            # Replacement Dataframe
    dwe,           # Dog Whistle Expression
    meaning,       # 1 for ingroup, 2 for outgroup
    phase,         # 1 for first phase of data collection, 2 for second phase
    sw = None,     # stopwords
    punct = None,  # remove punctuations
    verbose = True,
    multi = False  # Keep the multi-word units of the replacements
):
    
    counter = Counter()
    
    column = df.loc[df[f"{dwe}_w{phase}_C"] == meaning, f"{dwe}_text_w{phase}"]
    
    for x in column:
        if punct != None:
            for p in punct:
                x = x.replace(p, "")
        x = x.split()
        if sw != None:
            x = [w for w in x if w not in sw]
        
        if multi:
            x = ["_".join(x)]
        
        counter.update(x)
    
    if verbose:
        for w, f in sorted(counter.items(), key = lambda x: x[1], reverse = True)[:15]:
            print(f"{w:<20}{f}")
        print("-----------------------")
        print("Total no. of types:", len(counter))

    return counter

In [11]:
def keyness(trg, ref, min_frq = 3, verbose = True): # Consider metric
    
    d = dict()
    
    trg_tot = sum(trg.values())
    ref_tot = sum(ref.values())
    
    for w in trg.keys():
        if trg[w] < 3:
            continue
        if w in ref:
            d[w] = (trg[w] / trg_tot) / (ref[w] / ref_tot)
        else:
            d[w] = np.inf
    
    if verbose:
        for word, trg_freq, keyness  in sorted([(w, trg[w], k) for w, k in d.items()], key = lambda x: x[1], reverse = True)[:20]:
            if word in ref:
                ref_freq = ref[word]
            else:
                ref_freq = 0
            print(f"{word:<20}{trg_freq:<4}{(trg_freq/len(trg)):<6.3f}{ref_freq:<4}{(ref_freq/len(ref)):<6.3f}{keyness:.4}")        
    
    return d
    

In [12]:
def other(df, dwe, meaning, phase, criteria, verbose = True):
    
    oth = []
    
    column = df.loc[df[f"{dwe}_w{phase}_C"] == meaning, f"{dwe}_text_w{phase}"]

    for x in column:
        keep = True
        for criterion in criteria:
            if criterion in x:
                keep = False
                continue
        if keep:
            oth.append(x)
    
    if verbose:
        for x in oth:
            print(x)
    
    return oth


In [13]:
def inspect_all(
    df,            # Replacement Dataframe
    dwe,           # Dog Whistle Expression
    phase,         # 1 for first phase of data collection, 2 for second phase
    sw = None,     # stopwords
    punct = None,  # remove punctuations
):
    
    print("Most frequent")
    print("=============")
    print("In-group:")
    print("(a) Split")
    ingroup = inspect(df, dwe, 1, phase, sw, punct)
    print()
    print("(b) Keep phrases")
    _ = inspect(df, dwe, 1, phase, sw, punct, multi = True)
    print()
    print("Out-group:")
    print("(a) Split")
    outgroup = inspect(df, dwe, 2, phase, sw, punct)
    print()
    print("(b) Keep phrases")
    _ = inspect(df, dwe, 2, phase, sw, punct, multi = True)
    print()
    
    print("Keyness")
    print("=======")
    print("(a) in-group --> out-group")
    _ = keyness(ingroup, outgroup)
    print()
    
    print("(b) out-group --> in-group")
    _ = keyness(outgroup, ingroup)
    print()

    

In [14]:
def check_models():
    # path to models
    # path to frequency lists
    # list of terms as bases
    
    # ... in relation to time
    # forms with base in model
    # similarity of forms (with shared base)
    # frequency
    pass

##### Development

In [15]:
models_at = Path("/home/max/Results/fb_pol-yearly-rad3/models")
vocabs_at = Path("/home/max/Corpora/flashback-pol-time/yearly/fb-pt-radical3-v0/vocab")

In [ ]:
#os.listdir(models_at)

In [ ]:
model = "2004.w2v"
wv = KeyedVectors.load_word2vec_format(models_at / model)

In [ ]:
len(wv)

In [ ]:
#wv[1]

In [ ]:
"sverige" in wv

In [ ]:
#wv["sverige"]

In [ ]:
[w for w in wv.key_to_index if w.startswith("invandr")]

In [ ]:
# get_mean_vector(keys, weights=None, pre_normalize=True, post_normalize=False, ignore_missing=True)
wv.get_mean_vector(['invandrare', 'invandrarna'], weights=[0.7, 0.3], pre_normalize=True, post_normalize=False, ignore_missing=True)

In [ ]:
wv.similarity('invandrare', 'invandrarna')

In [ ]:
wv.n_similarity(['invandrare'], ['invandrarna', 'invandrarnas'])

In [ ]:
wv.similarity('invandrare', 'invandrargäng')

In [ ]:
wv.similarity('invandrare', 'människor')

In [ ]:
wv.most_similar("V1_berika")

In [ ]:

igt = [
    "förstöra",
    "utnyttja",
    "förpesta",
    "försämra", 
    "brott",
    "fördärva"
    ]
ogt = [
    "förbättra",
    "tillföra",
    "bidra",
    "förgylla",
    "nytta",
    "hjälpa", 
    "främja",
    "bättre",
    "utveckla",
    "stärka",
    "gynna",
    "förhöja",
    "positiv"
    ]
wv.n_similarity(["V1_berika"], igt), wv.n_similarity(["V1_berika"], ogt)

##### Association metric

In [48]:
def association(
    trg,        # Targets (list)
    igt,        # In-group terms
    ogt,        # Out-group terms
    kwv,        # Keyed word vector (`gensim.models.KeyedVectors`)
    method = "centroid", # "centroid" for similarity of averaged vector (centroid); 
                         # "mean" for mean of similarity of each comparison
    weigths_trg = None,  # weigths of trg
    weigths_igt = None,  # weigths of igt
    weigths_ogt = None,  # weigths of ogt
):
    
    if method == "centroid":
        
        trg_vec = kwv.get_mean_vector(trg, weights=weigths_trg, pre_normalize=True, post_normalize=False, ignore_missing=True)
        igt_vec = kwv.get_mean_vector(igt, weights=weigths_igt, pre_normalize=True, post_normalize=False, ignore_missing=True)
        ogt_vec = kwv.get_mean_vector(ogt, weights=weigths_ogt, pre_normalize=True, post_normalize=False, ignore_missing=True)
        
        igs = cosine_similarity([trg_vec, igt_vec])[0][1] # returns a matrix! must be indexed
        ogs = cosine_similarity([trg_vec, ogt_vec])[0][1]
        
        t = None
        p = None

    if method == "mean":
        
        igscores = []
        ogscores = []
        
        for trg_i in trg:      # What about OOV items? What do those calculations return?
            for igt_i in igt:
                ig_scores.append(kwv.similarity(trg_i, igt_i))
        
        for trg_i in trg:
            for ogt_i in ogt:
                og_scores.append(kwv.similarity(trg_i, ogt_i))
        
        igs = np.mean(igscores) # list to numpy array? axis = ? Check!
        ogs = np.mean(ogscores)
        
        t, p, df = ttest_ind(igscores, ogscores) # it is not perfectly clear that a t-test is a good idea here
        
    return igs, ogs, t, p
    
    

In [ ]:
##### Test: återvandring

In [47]:
cosine_similarity([[1,2,2], [2,33,1]])

array([[1.        , 0.70545306],
       [0.70545306, 1.        ]])

In [50]:
igt = [
        "utvisa", 
        "skicka", 
        "deportering", 
        "tvångsförflyttning", 
        "kasta", 
        "sända", 
        "tvinga", 
        "slänga", 
        "avvisa",
        "sparka",
        "avhysa"
]
ogt = [
    "återvända",
    "återreas",
    "repatriering",
    "hemresa",
    "hemåtvända",
    "utvandring",
    "flytta"
    ]
trg = ["N1_återvandring"]
for file in sorted([file for file in os.listdir(models_at) if file.endswith(".w2v")]):
    
    wv = KeyedVectors.load_word2vec_format(models_at / file)
    
    print(file, association(trg, igt, ogt, wv))

2000.w2v (0.0, 0.0, None, None)
2001.w2v (0.0, 0.0, None, None)
2002.w2v (0.0, 0.0, None, None)
2003.w2v (0.0, 0.0, None, None)
2004.w2v (0.0, 0.0, None, None)
2005.w2v (0.0, 0.0, None, None)
2006.w2v (0.0, 0.0, None, None)
2007.w2v (0.51223254, 0.70855886, None, None)
2008.w2v (0.49382836, 0.60381085, None, None)
2009.w2v (0.48515734, 0.6269076, None, None)
2010.w2v (0.4104982, 0.6042067, None, None)
2011.w2v (0.43976286, 0.6361747, None, None)
2012.w2v (0.45990908, 0.66302633, None, None)
2013.w2v (0.4576573, 0.68861943, None, None)
2014.w2v (0.43446374, 0.68996215, None, None)
2015.w2v (0.40961573, 0.6439194, None, None)
2016.w2v (0.41941243, 0.6422344, None, None)
2017.w2v (0.32260302, 0.6283942, None, None)
2018.w2v (0.43465668, 0.5938407, None, None)
2019.w2v (0.4029918, 0.59719825, None, None)
2020.w2v (0.4010749, 0.60859996, None, None)
2021.w2v (0.48089296, 0.62195784, None, None)
2022.w2v (0.4501851, 0.6430142, None, None)


## Explorations

### (1) Förortsgäng

#### Phase 1

In [17]:
inspect_all(replacement_data, "forortsgang", 1, stopwords, [","])

Most frequent
In-group:
(a) Split
invandrargäng       40
invandrare          32
invandrarungdomar   15
kriminella          11
invandrarkillar     9
ungdomar            9
gäng                8
unga                8
invandrarbakgrund   7
områden             5
förorten            4
invandrar           4
invandrartäta       4
grupp               3
invandrarungar      3
-----------------------
Total no. of types: 106

(b) Keep phrases
invandrargäng       32
invandrare          15
invandrarungdomar   13
invandrarkillar     4
kriminella_invandrare3
invandrar_gäng      3
unga_invandrarkillar3
kriminella_invandrargäng2
invandrarungar      2
invandrarkillar_utsatta_områden1
blattar             1
kriminella_invandarungdomar1
invandrargäng...    1
invandrarungdomar_utanförskapsområden1
gäng_invandrarbakgrund/kriminella_barn_ungdomar1
-----------------------
Total no. of types: 72

Out-group:
(a) Split
kriminella          158
gäng                107
ungdomar            101
grupper             88
ut

##### Ingroup terms

In [18]:
igt = [
        "invandr", 
        "blatt", 
        "utländsk", 
        "nysvensk", 
        "indvad",     # Miss-spelling ...
        "invandar", 
        "invadra", 
        "innvandr", 
        "indvandr"
    ]
_ = other(
    df = replacement_data, 
    dwe = "forortsgang", 
    meaning = 1, 
    phase = 1, 
    criteria = igt
)

lösdrivande ungdomar i klunga, överrepresenterade av icke svenskar
grupp boendes i förorten
många unga ej etiska svenskar i grupp
människor som rasifieras
yngre män, oftast med mellanöstern- eller nordafrikanskt ursprung boende i förort
grupper med ungdomar som har icke svenskt ursprung
kriminella med utomnordiskt påbrå i utsatta områden
subkulturella utmanar den etablerade ordningen


##### Outgroup terms

Hard! See above!

#### Phase 2

In [19]:
inspect_all(replacement_data, "forortsgang", 2, stopwords, [","])

Most frequent
In-group:
(a) Split
invandrare          121
invandrargäng       112
kriminella          38
invandrarungdomar   30
unga                28
ungdomar            21
gäng                19
invandrarbakgrund   17
invandrar           14
grupper             14
förorten            12
invandrarkillar     12
killar              10
bakgrund            10
utländsk            10
-----------------------
Total no. of types: 296

(b) Keep phrases
invandrargäng       94
invandrare          55
invandrarungdomar   26
invandrarkillar     7
invandrargrupper    6
kriminella_invandrare5
invandrar_ungdomar  4
utlänningar         4
unga_invandrare     3
nysvenskar          3
invandrarungdommar  3
kriminella_invandrargäng3
invandrarungdom     3
personer_invandrarbakgrund3
barn_invandrare     3
-----------------------
Total no. of types: 216

Out-group:
(a) Split
kriminella          554
ungdomar            305
gäng                286
grupper             218
utsatta             192
områden            

##### Ingroup terms

In [20]:
igt = [
        "invandr", 
        "blatt", 
        "babb",
        "svartskall",
        "neger",
        "utlänning", 
        "utlandsfödd", 
        "ickesvensk",
        "nysvensk",
        "arab",
        "utländsk",
        "utomnordisk",
        "utomeuropeisk",
        "invanda",        # Miss-spelling
        "invadr",
        "ivandr"
    ] 
_ = other(
    df = replacement_data, 
    dwe = "forortsgang", 
    meaning = 1, 
    phase = 2, 
    criteria = igt
)

kriminella gäng
ungdomar från miljonprogrammen
organiserad brottslighet
lokala ungdomsgrupper
ligister
ungdomsskaror från de yttre stadsdelarna
gäng från icke integrerbara kulturer
etniska ligor
kulturberikare
etniska grupper utanför samhället
kriminella grupperingar
urbana grupperingar
rasfrämlingar
gängkriminella
som gw säger: inget svenne banan problem
icke svenskar i förorter
"kulturella skillnader ligger bakom trakasserier, hot, skadegörelse och våld mot poliser."
kulturberikare
gäng mest bestående av människor av annat ursprung än svenskt
män med utrikes bakgrund
unga med utrikes bakgrund
gäng av kriminella ffa yngre män med rötter i mellanöstern och afrika
grupp med utlandssvenskar
ungdomsgäng i närförort med blandade ursprung.
dåligt integrerade, antisociala grupperingar med kriminella alternativt hedersrelaterade motiv
grupperingar i förorten, grupper som känner sig exkludera. rasismen som myntar ny begrep
grupper som hålls samman av att de lever i storstäders utkanter och kan

##### Outgroup terms

Hard! See above!

### (2) Återvandring

#### Phase 1

In [36]:
Counter(replacement_data['aterinvandring_w1_C'])

Counter({nan: 2437, '1': 210, '2': 594})

In [42]:
list(replacement_data.columns)

['forortsgang_text_w1',
 'forortsgang_w1_C',
 'forortsgang_text_w2',
 'forortsgang_w2_C',
 'aterinvandring_text_w1',
 'aterinvandring_w1_C',
 'aterinvandring_text_w2',
 'aterinvandring_w2_C',
 'berikar_text_w1',
 'berikar_w1_C',
 'berikar_text_w2',
 'berikar_w2_C',
 'globalister_text_w1',
 'globalister_w1_C',
 'globalister_text_w2',
 'globalister_w2_C',
 'ordningreda_text_w1',
 'ordningreda_w1_C',
 'ordningreda_text_w2',
 'ordningreda_w2_C',
 'sjalvstandig_text_w1',
 'sjalvstandig_w1_C',
 'sjalvstandig_text_w2',
 'sjalvstandig_w2_C']

In [43]:
# WHAT IS WRONG?
inspect_all(replacement_data, "aterinvandring", 1, sw = stopwords, punct = [","])

Most frequent
In-group:
(a) Split
-----------------------
Total no. of types: 0

(b) Keep phrases
-----------------------
Total no. of types: 0

Out-group:
(a) Split
-----------------------
Total no. of types: 0

(b) Keep phrases
-----------------------
Total no. of types: 0

Keyness
(a) in-group --> out-group

(b) out-group --> in-group



##### Ingroup terms

In [23]:
igt = [
        "utvis", 
        "skick", 
        "deport", 
        "tvångsförflyttning", 
        "kast", 
        "sänd", 
        "tvinga", 
        "släng", 
        "avvis"
    ]
_ = other(
    df = replacement_data, 
    dwe = "aterinvandring", 
    meaning = 1, 
    phase = 1, 
    criteria = igt
)

*Comment:* wrong code? återvändande, tillbakaflytt, repatriering, hemvändande

##### Outgroup terms

In [24]:
ogt = [
    "hemvänd",
    "återvänd",
    "återres",
    "repatrier",
    "hemres",
    "hemåtvända",
    "utvandring",
    "flytt"
    ]
_ = other(
    df = replacement_data, 
    dwe = "aterinvandring", 
    meaning = 2, 
    phase = 1, 
    criteria = ogt
)

*Comment:* wrong code? On several. utvisning, etnisk rensning, skicka tillbaka, avvisa

#### Phase 2

In [26]:
inspect_all(replacement_data, "aterinvandring", 2, sw = stopwords, punct = [","])

Most frequent
In-group:
(a) Split
utvisning           224
skicka              110
tillbaka            80
invandrare          59
hem                 43
deportering         37
människor           32
hemländer           26
folk                21
flyktingar          19
tvinga              16
utvisa              15
uppehållstillstånd  14
utvisningar         14
tillbaks            13
-----------------------
Total no. of types: 415

(b) Keep phrases
utvisning           171
deportering         31
utvisningar         14
skicka_hem          12
skicka_hem_invandrare10
deportation         8
skicka_tillbaka     7
skicka_tillbaka_invandrare_hemländer5
etnisk_rensning     5
utvisning_invandrare5
utvisa              5
skicka_tillbaka_invandrare5
sparka              4
avvisning           4
rasism              4
-----------------------
Total no. of types: 322

Out-group:
(a) Split
tillbaka            466
flytta              329
återvända           213
invandrare          184
hemland             165
heml

##### Ingroup terms

In [28]:
igt = [
        "utvis", 
        "skick", 
        "deport", 
        "tvångsförflyttning", 
        "kast", 
        "sänd", 
        "tvinga", 
        "släng", 
        "avvis",
        "spark",
        "avhys",
        "förvis"
    ]
_ = other(
    df = replacement_data, 
    dwe = "aterinvandring", 
    meaning = 1, 
    phase = 2, 
    criteria = igt
)

färre uppehållstillstånd
frampressad flytt
vi vill inte ha folk av andra hudfärger här, för vi är fega rasister
etnisk rensning
att dra tillbaka folks uppehållstillstånd
racism
"frivilligt" återvändande till hemlandet
vägran inträde
etnisk rensning
tvång att återvända till krig och förtryck
att med hjälp av morot eller piska få invandrare att lämna sverige för sitt ursprungsland
fördrivning
fascism
forcerad utvandring till krigszoner
tvångsåtgärder för att få folk att återvända till sina hemländer
gränskontroll
export, utvsndring, återställning
det är redan en snäll omskrivning för "stick härifrån"
fördrivning
nekad asyl
rasism
nekande av asyl
minskning av ej etniska svenskar
etnisk rensning
neka immigranter
borttryckning
i bästa fall reser människor tillbaka till sina hemländer, i värsta fall är det rasismen som påverka
att få personer att flytta tillbaka till de länder de föddes i, med eller mot deras vilja,
etnisk rensning
att påverka invandrare att återvända, med morot eller piska


*Comment:* wrong code? 

##### Outgroup terms

In [29]:
ogt = [
    "hemvänd",
    "återvänd",
    "återres",
    "repatrier",
    "hemres",
    "hemåtvända",
    "utvandring",
    "flytt"
    ]
_ = other(
    df = replacement_data, 
    dwe = "aterinvandring", 
    meaning = 2, 
    phase = 2, 
    criteria = ogt
)

returresa
att låta invandrare åka hem
uppmuntran att resa tillbaka till hemlandet
emigration
tillbakablick
exportera
åka till bala
reversering
se vilka möjligheter som finns att kunna acceptera ursprungslandets (tidigare hemlandets) levnadssät
att åka tillbaka
respektera, acceptera
icke permanenta uppehållstillstånd
de som inte ska eller får vara här pga lagar
resa tillbaks till ursprungslandet
utvisning på frivillig basis
att förmå personer att resa tillbaka till sina ursprungsländer
migration tillbaka till ursprungslandet
att vända tillbaka till ursprungs landet
tillbaka till hemlandet
återvandring
reintegrering i hemlandet
hjälpa dem att åka tillbaka
dålig migration löses inte av dålig återvandring
tillbakagående
att få asylinvandrare och kvotflyktingar
medborgarskap och utbildning
tillbaka till sina hemländer
åka tillbaka
att invandrare lämnar sverige
återsändning till hemlandet
minskning av antalet utifrån kommande
att få åka hem till sitt ursprungsland lyfts fram som ett sätt att

*Comment:* wrong code? förpassning

### (3) Berikar

#### Phase 1

In [ ]:
inspect_all(replacement_data, "berikar", 1, sw = stopwords, punct = [","])

##### Ingroup terms

In [ ]:
igt = [
    "förstör",
    "utnyttja",
    "förpest",
    "försämra", 
    "brott",
    "fördärv"
    ]
_ = other(
    df = replacement_data, 
    dwe = "berikar", 
    meaning = 1, 
    phase = 1, 
    criteria = igt
)

##### Outgroup terms

In [ ]:
ogt = [
    "förbättra",
    "tillför",
    "bidra",
    "förgyll",
    "nytta",
    "hjälp", 
    "främja",
    "bättre",
    "utveckla",
    "stärk",
    "gynna",
    "förhöj",
    "positiv"
    ]
_ = other(
    df = replacement_data, 
    dwe = "berikar", 
    meaning = 2, 
    phase = 1, 
    criteria = ogt
)

#### Phase 2

In [ ]:
inspect_all(replacement_data, "berikar", 2, sw = stopwords, punct = [","])

##### Ingroup terms

In [ ]:
igt = [
    "förstör",
    "utnyttja",
    "förpest",
    "försämra", 
    "brott",
    "fördärv"
    ]
_ = other(
    df = replacement_data, 
    dwe = "berikar", 
    meaning = 1, 
    phase = 1, 
    criteria = igt
)

##### Outgroup terms

In [ ]:
ogt = [
    "förbättra",
    "tillför",
    "bidra",
    "förgyll",
    "nytta",
    "hjälp", 
    "främja",
    "bättre",
    "utveckla",
    "stärk",
    "gynna",
    "förhöj", 
    "positiv"
    ]
_ = other(
    df = replacement_data, 
    dwe = "berikar", 
    meaning = 2, 
    phase = 1, 
    criteria = ogt
)

### (4) Globalister

#### Phase 1

In [ ]:
inspect_all(replacement_data, "globalister", 1, sw = stopwords, punct = [","])

##### Ingroup terms

In [ ]:
igt = [
    "judar",
    "elit"
    ]
_ = other(
    df = replacement_data, 
    dwe = "globalister", 
    meaning = 1, 
    phase = 1, 
    criteria = igt
)

##### Outgroup terms

In [ ]:
ogt = [
    "internationell" # ?

    ]
_ = other(
    df = replacement_data, 
    dwe = "globalister", 
    meaning = 2, 
    phase = 1, 
    criteria = ogt
)

#### Phase 2

In [ ]:
inspect_all(replacement_data, "globalister", 2, sw = stopwords, punct = [",", "."])

##### Ingroup terms

In [ ]:
igt = [
    "judar",
    "judisk",
    "soro",
    "elit"
    ]
_ = other(
    df = replacement_data, 
    dwe = "globalister", 
    meaning = 1, 
    phase = 2, 
    criteria = igt
)

##### Outgroup terms

In [ ]:
ogt = [
    "internationell", # ?
    "värld"

    ]
_ = other(
    df = replacement_data, 
    dwe = "globalister", 
    meaning = 2, 
    phase = 2, 
    criteria = ogt
)

*Comment:* Polysemous dogwhistle... 
* landsförrädare, landsförädare, icke-nationalister; 
* kapitaliser; 
* liberalister; 
* sossar